In [45]:


class Graph:    
    # -------------------------- nested Edge class ------------------------------------
    class Edge:
        """Lightweight edge structure for a graph."""
        __slots__ = '_origin', '_destination', '_element'
    
        def __init__(self, u, v, x):
            """Do not call constructor directly. Use Graph's insert_edge(u, v, x)"""
            self._origin = u 
            self._destination = v 
            self._element = x 
        
        def endpoints(self):
            """Return (u, v) tuple for vertices u and v."""
            return (self._origin, self._destination)
        
        def opposite(self, v):
            """Return the vertex that is opposite v on this edge."""
            return self._destination if v is self._origin else self._origin 
    
        def element(self):
            """Return element associated with this edge"""
            return self._element 
        
        def __hash__(self):      # will allow edge to be a map/set key
            return hash((self._origin, self._destination))

    # ----------------------- nested Vertex class ----------------------------------------
    class Vertex:
        """Lightweight vertex structure for a graph."""
        __slots__ = '_element'

        def __init__(self, x):
            """Do not call constructor directly. Use Graph's insert_vertex(x)."""
            self._element = x 
        
        def element(self):
            """Return element associated with this vertex."""
            return self._element 

        def __hash__(self):   # will allow vertex to be a map/set key
            return hash(id(self))
   
    """Representation of a simple graph using an adjacency map."""
    def __init__(self, directed=False):
        """Create an empty graph (undirected, by default).
        Graph is directed if optional paramter is set to True.
        """
        self._outgoing = {}
        self._incoming = {} if directed else self._outgoing 
    
    def is_directed(self):
        """Return True if this is a directed graph; False if undirected.
        Property is based on the original declaration of the graph, not its contents.
        """
        return self._incoming is not self._outgoing 

    def vertex_count(self):
        """Return an iteration of all vertices of the graph."""
        return self._outgoing.keys()
    
    def edge_count(self):
        """Return the number of edges in the graph."""
        total = sum(len(self._outgoing[v]) for v in self._outgoing)
        # for undirected graphs, make sure not to double-count edges 
        return total if self.is_directed() else total // 2
    
    def edges(self):
        """Return a set of all edges of the graph."""
        result = set()            # avoid double-reporting edges of undirected graph
        for secondary_map in self._outgoing.values():
            result.update(secondary_map.values())
        return result 

    def vertices(self):
        """Return an iteration of all vertices of the graph."""
        return self._outgoing.keys()

    def get_edge(self, u, v):
        """Return the edge form u to v, or None if not adjacent."""
        print('v is ->', u.element())
        
        return self._outgoing[u].get(v)
    
    def degree(self, v, outgoing=True):
        """Return number of (outgoing) edges incident to verte v in the graph.
        If graph is direceted, optional parameter used to count incoming edges.
        """
        adj = self._outgoing if outgoing else self._incoming 
        return len(adj[v])
    
    def incident_edges(self, v, outgoing=True):
        """Return all (outgoing) edges incident to vertex v in the graph.
        If graph is direceted, optional parameter used to count incoming edges.
        """
        adj = self._outgoing if outgoing else self._incoming 
        for edge in adj[v].values():
            yield edge 
    
    def insert_vertex(self, x = None):
        """Insert and return a new Vertex with element x."""
        v = self.Vertex(x)
        self._outgoing[v] = {}
        if self.is_directed():
            self._incoming[v] = {}         # need distinct map for incoming edges 
        return v 
    
    def insert_edge(self, u, v, x=None):
        """Insert and return a new Edge from u to v with auxiliary element x."""
        e = self.Edge(u, v, x)
        self._outgoing[u][v] = e
        print('here', [e.element() for e in self._outgoing[u].keys()])
        self._incoming[v][u] = e 

    def DFS(self, g, u, discovered: dict = dict()):
        """Perform DFS of the undiscovered portion of Graph g starting at Vertex u.
            discovered is a dictionary mapping each vertex to the edge that was used to 
            discover it during the DFS. (u should be "discovered" prior to call.)
            Newly discovered vertices will be added to the dictionary as a result.
        """
        for e in g.incident_edges(u):           # for every outgoing edge from u 
            v = e.opposite(u)
            if v not in discovered:            # v is an unvisited vertex 
                discovered[v] = e              # e is the tree edge that discovered v 
                self.DFS(g, v, discovered)
        return discovered 

    def construct_path(self, u, v, discovered):
        path = []                             # empty path by default 
        if v in discovered:
            # we build list from v to u and then reverse it at the end.
            path.append(v)
            walk = v 
            while walk is not u:
                e = discovered[walk]          # find edge leading to walk
                parent = e.opposite(walk)
                path.append(parent)
                walk = parent 
            path.reverse()                    # reorient path from u to v 
        return path          
    
    def DFS_complete(self, g):
        """
        Perform DFS for entire graph and return forest as a dictionary.

        Result maps each vertex v to the edge that was used to discovered it. 
        (Vertices that are roots of a DFS tree are mapped to None.)
        """
        forest = {}
        for u in g.vertices():
            if u not in forest:
                forest[u] = None               # u will be the root of a tree 
                self.DFS(g, u, forest)
        return forest 

ga = Graph()
v_a = ga.insert_vertex('A')
v_b = ga.insert_vertex('B')
v_c = ga.insert_vertex('C')
v_d = ga.insert_vertex('D')
v_e = ga.insert_vertex('E')
v_f = ga.insert_vertex('F')
v_g = ga.insert_vertex('G')
v_h = ga.insert_vertex('H')
v_j = ga.insert_vertex('J')
ga.insert_edge(v_a, v_b, 'ab')
ga.insert_edge(v_b, v_c, 'bc')
ga.insert_edge(v_c, v_d, 'cd')
ga.insert_edge(v_d, v_e, 'de')
ga.insert_edge(v_c, v_e, 'ce')
ga.insert_edge(v_e, v_g, 'eg')
ga.insert_edge(v_c, v_f, 'cf')
ga.insert_edge(v_h, v_j, 'hj')


# ---------------------- Graph Traversal -------------------
"""
Algorithm DFS(G, u):             {We assume u has already been marked as visited}
    Input: A graph G and a vertex u of G 
    Output: A collection of vertices reachable from u with their discovery edges 
    
    for each outgoing edge e = (u,v) of u do 
        if vertex v has not been visited then 
            Mark vertex v as visited (via edge e).
            Recursively call DFS(G, v).
""" 
discovered = ga.DFS(ga, v_a, {v_a: None})
# construct path 
path = ga.construct_path(v_a, v_f, discovered)
print([v.element() for v in path])

# Computing all Connected Components 
# If an initial call to DFS fails to reach all vertices of a graph, we can restart a 
# new call to DFS at on of those unvisited vertices. 
print({key.element():val for key, val in ga.DFS_complete(ga).items()})


here ['B']
here ['A', 'C']
here ['B', 'D']
here ['C', 'E']
here ['B', 'D', 'E']
here ['D', 'C', 'G']
here ['B', 'D', 'E', 'F']
here ['J']
['A', 'B', 'C', 'F']
{'A': None, 'B': <__main__.Graph.Edge object at 0x10792ad80>, 'C': <__main__.Graph.Edge object at 0x1076299c0>, 'D': <__main__.Graph.Edge object at 0x106f36b80>, 'E': <__main__.Graph.Edge object at 0x10792af80>, 'G': <__main__.Graph.Edge object at 0x1076c78c0>, 'F': <__main__.Graph.Edge object at 0x1076c7dc0>, 'H': None, 'J': <__main__.Graph.Edge object at 0x1076c7180>}


KeyError: 0